<a href="https://colab.research.google.com/github/OfficalOffical/BasicTranslator/blob/master/randomFOOOOOOOREST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.src.regularizers import l1_l2, l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Conv1D, BatchNormalization, Activation, MaxPooling1D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.src.saving.saving_api import load_model
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt
import joblib  # Import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from joblib import dump

import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt
import joblib  # Import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from joblib import dump

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
import re


def work_on_outliers(input_df,inf_threshold,sup_threshold, feature_number,visualize):
    # WORKING ON OUTLIERS

    # Working on negpmax feature (the only one that is showing outliers)
    features = [f for f in input_df.columns[feature_number::4]]
    if visualize:
        fig, axes = plt.subplots(4, 3, figsize=(15, 10))  # Adjust the figsize as needed

        # Flatten the 2D array of subplots
        axes = axes.flatten()

    output_df=input_df.copy()

    for i, feature in enumerate(features):
        Q1 = input_df[feature].quantile(0.25)
        Q3 = input_df[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - inf_threshold * IQR

        if feature_number==3:
            upper_bound=0
        else:
            upper_bound=Q3 + sup_threshold * IQR




        # removing too  small values and positive values for negpmax
        #removing too large values for rms

        outliers = output_df[(output_df[feature] < lower_bound) | (output_df[feature] > upper_bound)]



        output_df = output_df[(output_df[feature] > lower_bound) & (output_df[feature] < upper_bound)]
        if visualize:
            axes[i].scatter(outliers["x"], outliers["y"], c=outliers[feature], cmap='viridis')  # Adjust the colormap as needed
            axes[i].set_title(f'REMOVED OUTLIERS: {feature}')
            axes[i].set_xlabel("X-axis")
            axes[i].set_ylabel("Y-axis")

    if visualize:
        plt.tight_layout()
        plt.show()


    if visualize:

        # let's see if the representation is working better after removing the outliers '
        random_10_percent = output_df.sample(frac=0.1, random_state=42)

        fig, axes = plt.subplots(4, 3, figsize=(15, 10))  # Adjust the figsize as needed
        fig.suptitle("RESULT AFTER REMOVING OUTLIERS (ON A 10% SAMPLE) ", fontsize=16)
        # Flatten the 2D array of subplots
        axes = axes.flatten()

        for i, feature in enumerate(features):
            axes[i].scatter(random_10_percent["x"], random_10_percent["y"], c=random_10_percent[feature],
                            cmap='viridis',label=feature)  # Adjust the colormap as needed
            axes[i].set_title(f'scatter plot: {feature}')
            axes[i].set_xlabel("X-axis")
            axes[i].set_ylabel("Y-axis")
            axes[i].legend(loc='best')  # Add legend

        # Adjust layout to prevent overlap
        plt.tight_layout()
        plt.show()


    return output_df


#MAIN CODE
def pre_process_removing_area_keeping_outliers(df,ev=False, normalize=False, see_graphs=False):
    #REMOVING NOISE COLUMNS

    #working on tmax features, which seem to be the most meaningful for removing the noise sensors
    if ev==False:
        z_scores = pd.DataFrame(zscore(df), columns=df.columns[5::5])
    else:
        z_scores = pd.DataFrame(zscore(df), columns=df.columns[4::5])
    outlier_threshold = 3
    outliers = (z_scores.abs() > outlier_threshold)

    #noise sensors have a random distribution of tmax, almost all  the values are contained
    #in the value of 3*dev_std. We order by increasing order of outliers and remove the columns
    #that have less outliers (random distribution)

    percentage_outliers = (outliers.sum() / len(df)) * 100
    sorted_columns = percentage_outliers.sort_values()
    numbers_to_remove = [re.search(r'\[(\d+)\]', column).group(1) for column in sorted_columns[:6].index]
    columns_to_drop = [column for column in df.columns if
                       any(re.search(rf'\[{number}\]', column) for number in numbers_to_remove)]

    df_clean = df.drop(columns=columns_to_drop)

    # the feature describing the area and the feature describing the pmax are highly correlated,
    # we can remove the area feature.
    if ev==False:
        features_area = [f for f in df_clean.columns[4::5]]
    else:
        features_area = [f for f in df_clean.columns[3::5]]

    df_clean = df_clean.drop(columns=features_area)

    return df_clean


def pre_process_removing_area_removing_outliers(df,ev=False, normalize=False, see_graphs=False):
    #REMOVING NOISE COLUMNS

    #working on tmax features, which seem to be the most meaningful for removing the noise sensors
    if ev==False:
        z_scores = pd.DataFrame(zscore(df), columns=df.columns[5::5])
    else:
        z_scores = pd.DataFrame(zscore(df), columns=df.columns[4::5])
    outlier_threshold = 3
    outliers = (z_scores.abs() > outlier_threshold)

    percentage_outliers = (outliers.sum() / len(df)) * 100
    sorted_columns = percentage_outliers.sort_values()
    numbers_to_remove = [re.search(r'\[(\d+)\]', column).group(1) for column in sorted_columns[:6].index]
    columns_to_drop = [column for column in df.columns if
                       any(re.search(rf'\[{number}\]', column) for number in numbers_to_remove)]

    df_clean = df.drop(columns=columns_to_drop)


    if ev==False:
        features_area = [f for f in df_clean.columns[4::5]]
    else:
        features_area = [f for f in df_clean.columns[3::5]]

    df_clean = df_clean.drop(columns=features_area)

    #OUTLIERS

    if ev==False:
        df_clean_out_negpmax=work_on_outliers(df_clean,25,0,3,see_graphs) #3 means negpmax
        df_clean=df_clean_out_negpmax.copy()
        #we can work also on the rms

        df_clean_out_rms = work_on_outliers(df_clean, 1.5,1.5, 5,see_graphs)  # 5 means rms
        df_clean=df_clean_out_rms.copy()




    return df_clean


def pre_process_keeping_area_removing_outliers(df, ev=False, normalize=False, see_graphs=False):
    # REMOVING NOISE COLUMNS

    # working on tmax features, which seem to be the most meaningful for removing the noise sensors
    if ev == False:
        z_scores = pd.DataFrame(zscore(df), columns=df.columns[5::5])
    else:
        z_scores = pd.DataFrame(zscore(df), columns=df.columns[4::5])
    outlier_threshold = 3
    outliers = (z_scores.abs() > outlier_threshold)

    # noise sensors have a random distribution of tmax, almost all  the values are contained
    # in the value of 3*dev_std. We order by increasing order of outliers and remove the columns
    # that have less outliers (random distribution)

    percentage_outliers = (outliers.sum() / len(df)) * 100
    sorted_columns = percentage_outliers.sort_values()
    numbers_to_remove = [re.search(r'\[(\d+)\]', column).group(1) for column in sorted_columns[:6].index]
    columns_to_drop = [column for column in df.columns if
                       any(re.search(rf'\[{number}\]', column) for number in numbers_to_remove)]

    df_clean = df.drop(columns=columns_to_drop)


    # OUTLIERS

    if ev == False:
        df_clean_out_negpmax = work_on_outliers(df_clean, 25, 0, 3, see_graphs)  # 3 means negpmax
        df_clean = df_clean_out_negpmax.copy()
        # we can work also on the rms

        df_clean_out_rms = work_on_outliers(df_clean, 1.5, 1.5, 5, see_graphs)  # 5 means rms
        df_clean = df_clean_out_rms.copy()

    return df_clean



def pre_process_keeping_area_keeping_outliers(df, ev=False, normalize=False, see_graphs=False):
    # REMOVING NOISE COLUMNS

    # working on tmax features, which seem to be the most meaningful for removing the noise sensors
    if ev == False:
        z_scores = pd.DataFrame(zscore(df), columns=df.columns[5::5])
    else:
        z_scores = pd.DataFrame(zscore(df), columns=df.columns[4::5])
    outlier_threshold = 3
    outliers = (z_scores.abs() > outlier_threshold)

    # noise sensors have a random distribution of tmax, almost all  the values are contained
    # in the value of 3*dev_std. We order by increasing order of outliers and remove the columns
    # that have less outliers (random distribution)

    percentage_outliers = (outliers.sum() / len(df)) * 100
    sorted_columns = percentage_outliers.sort_values()
    numbers_to_remove = [re.search(r'\[(\d+)\]', column).group(1) for column in sorted_columns[:6].index]
    columns_to_drop = [column for column in df.columns if
                       any(re.search(rf'\[{number}\]', column) for number in numbers_to_remove)]

    df_clean = df.drop(columns=columns_to_drop)



    return df_clean












In [4]:
from tensorflow.keras.losses import Loss
import tensorflow.keras.backend as K
import pandas as pd
from keras import backend as K
import keras.saving
from keras.saving import load_model


@keras.saving.register_keras_serializable()
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

def euclidean_distance_error(y_true, y_pred):
    """
    Calculate the mean Euclidean distance error between true and predicted 2D points.
    :param y_true: Array of true values with shape (n_samples, 2).
    :param y_pred: Array of predicted values with shape (n_samples, 2).
    :return: Mean Euclidean distance error.
    """
    return np.mean(np.sqrt(np.sum(np.square(y_true - y_pred), axis=1)))



In [5]:


df = pd.read_csv('/content/drive/MyDrive/development.csv')


df = pre_process_removing_area_removing_outliers(df)

# Prepare the dataset
X = df.iloc[:, 2:].values
Y = df.iloc[:, :2].values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)




# Further split training set to create a validation set
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # Adjust the test_size as needed

# Reshape the data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

# Define the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='linear'))

# make the step of adam optimizer more smaller


# Compile the model
model.compile(optimizer='adam', loss=euclidean_distance_loss)

model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)

model.fit(X_train, Y_train, epochs=500, batch_size=32, validation_data=(X_val, Y_val), callbacks=[early_stopping,model_checkpoint], verbose=1)

# Evaluate the model
predictions = model.predict(X_test)

print(predictions)

mse = mean_squared_error(Y_test, predictions)
print(f"Mean Squared Error: {mse}")
rmse = sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")
euclidian= euclidean_distance_error(Y_test, predictions)
print(f"Euclidian distance Error: {euclidian}")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6272      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 2)                 66        
                                                                 
Total params: 16674 (65.13 KB)
Trainable params: 16674 (65.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500
6924/6924 [==============================] - 16s 2ms/step - loss: 15.3383 - val_loss: 11.0521
Epoch 2/500
6924/6924 [=======================

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Further split training set to create a validation set
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # Adjust the test_size as needed

# Reshape the data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

In [ ]:
model = Sequential([
    Conv1D(128, 3, input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(3),
    Dropout(0.2),


    Conv1D(64, 3),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(3),
    Dropout(0.2),




    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='linear')
])




model.compile(optimizer='adam', loss=euclidean_distance_loss)

model.summary()# assuming custom loss function

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('best_model1010.keras', save_best_only=True)



model.fit(X_train, Y_train, epochs=500, batch_size=32, validation_data=(X_val, Y_val), callbacks=[early_stopping,model_checkpoint], verbose=1)
# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print(f"Mean Squared Error: {mse}")
rmse = sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")
euclidian= euclidean_distance_error(Y_test, predictions)
print(f"Euclidian distance Error: {euclidian}")









Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 46, 128)           512       
                                                                 
 batch_normalization (Batch  (None, 46, 128)           512       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 46, 128)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 15, 128)           0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 15, 128)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 13, 64)           

In [ ]:
model = Sequential([





    Conv1D(64, 3, input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(3),







    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='linear')
])




model.compile(optimizer=Adam(learning_rate=0.001), loss=euclidean_distance_loss)

model.summary()# assuming custom loss function

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('best_model1002.keras', save_best_only=True)



model.fit(X_train, Y_train, epochs=550, batch_size=32, validation_data=(X_val, Y_val), callbacks=[early_stopping,model_checkpoint], verbose=1)
# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print(f"Mean Squared Error: {mse}")
rmse = sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")
euclidian= euclidean_distance_error(Y_test, predictions)
print(f"Euclidian distance Error: {euclidian}")









Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 46, 64)            256       
                                                                 
 batch_normalization_2 (Bat  (None, 46, 64)            256       
 chNormalization)                                                
                                                                 
 activation_2 (Activation)   (None, 46, 64)            0         
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 15, 64)            0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 960)               0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from math import sqrt
import joblib

# Data splitting (as before)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    colsample_bytree=0.8,    # Maintaining diversity
    learning_rate=0.016,     # Slightly reduced for more precision
    max_depth=12,            # Increased depth for capturing complexity
    alpha=14,                # Increased regularization to control overfitting
    n_estimators=3350,       # Slightly more trees for learning
    subsample=0.87,          # Fine-tuned subsample rate
    min_child_weight=3,      # Increased to reduce overfitting risk
    gamma=0.25,              # Adjusted for minimum loss reduction
    reg_lambda=2             # Increased L2 regularization
)


# Fit the model
model.fit(X_train, Y_train, early_stopping_rounds=20, eval_set=[(X_val, Y_val)], verbose=True)

# Save the model
joblib.dump(model, "improved_model.xgb")


# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print(f"Mean Squared Error: {mse}")
rmse = sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")
euclidian= euclidean_distance_error(Y_test, predictions)
print(f"Euclidian distance Error: {euclidian}")

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:114.85434
[1]	validation_0-rmse:113.03433
[2]	validation_0-rmse:111.24340
[3]	validation_0-rmse:109.48065
[4]	validation_0-rmse:107.74560
[5]	validation_0-rmse:106.03876
[6]	validation_0-rmse:104.35941
[7]	validation_0-rmse:102.70863
[8]	validation_0-rmse:101.08299
[9]	validation_0-rmse:99.48282
[10]	validation_0-rmse:97.90840
[11]	validation_0-rmse:96.35920
[12]	validation_0-rmse:94.83650
[13]	validation_0-rmse:93.33574
[14]	validation_0-rmse:91.85865
[15]	validation_0-rmse:90.40505
[16]	validation_0-rmse:88.97557
[17]	validation_0-rmse:87.56876
[18]	validation_0-rmse:86.18585
[19]	validation_0-rmse:84.82444
[20]	validation_0-rmse:83.48545
[21]	validation_0-rmse:82.16597
[22]	validation_0-rmse:80.86829
[23]	validation_0-rmse:79.59058
[24]	validation_0-rmse:78.33191
[25]	validation_0-rmse:77.09370
[26]	validation_0-rmse:75.87580
[27]	validation_0-rmse:74.67790
[28]	validation_0-rmse:73.49887
[29]	validation_0-rmse:72.33911
[30]	validation_0-rmse:71.19674
[31]	vali

In [6]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt
import joblib  # Import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from joblib import dump





X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# Model initialization and training with verbose output
model = RandomForestRegressor(n_estimators=850, random_state=42, verbose=3)
model.fit(X_train, Y_train)

dump(model, 'random_forest_model.joblib')

# Predicting the targets
predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print(f"Mean Squared Error: {mse}")
rmse = sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")
euclidian= euclidean_distance_error(Y_test, predictions)
print(f"Euclidian distance Error: {euclidian}")

building tree 1 of 850
building tree 2 of 850
building tree 3 of 850
building tree 4 of 850
building tree 5 of 850
building tree 6 of 850
building tree 7 of 850
building tree 8 of 850
building tree 9 of 850
building tree 10 of 850
building tree 11 of 850
building tree 12 of 850
building tree 13 of 850
building tree 14 of 850
building tree 15 of 850
building tree 16 of 850
building tree 17 of 850
building tree 18 of 850
building tree 19 of 850
building tree 20 of 850
building tree 21 of 850
building tree 22 of 850
building tree 23 of 850
building tree 24 of 850
building tree 25 of 850
building tree 26 of 850
building tree 27 of 850
building tree 28 of 850
building tree 29 of 850
building tree 30 of 850
building tree 31 of 850


[Parallel(n_jobs=1)]: Done  31 tasks      | elapsed:  9.7min


building tree 32 of 850
building tree 33 of 850
building tree 34 of 850
building tree 35 of 850
building tree 36 of 850
building tree 37 of 850
building tree 38 of 850
building tree 39 of 850
building tree 40 of 850
building tree 41 of 850
building tree 42 of 850
building tree 43 of 850
building tree 44 of 850
building tree 45 of 850
building tree 46 of 850
building tree 47 of 850
building tree 48 of 850
building tree 49 of 850
building tree 50 of 850
building tree 51 of 850
building tree 52 of 850
building tree 53 of 850
building tree 54 of 850
building tree 55 of 850
building tree 56 of 850
building tree 57 of 850
building tree 58 of 850
building tree 59 of 850
building tree 60 of 850
building tree 61 of 850
building tree 62 of 850
building tree 63 of 850
building tree 64 of 850
building tree 65 of 850
building tree 66 of 850
building tree 67 of 850
building tree 68 of 850
building tree 69 of 850
building tree 70 of 850
building tree 71 of 850
building tree 72 of 850
building tree 73

[Parallel(n_jobs=1)]: Done 127 tasks      | elapsed: 39.9min


building tree 128 of 850
building tree 129 of 850
building tree 130 of 850
building tree 131 of 850
building tree 132 of 850
building tree 133 of 850
building tree 134 of 850
building tree 135 of 850
building tree 136 of 850
building tree 137 of 850
building tree 138 of 850
building tree 139 of 850
building tree 140 of 850
building tree 141 of 850
building tree 142 of 850
building tree 143 of 850
building tree 144 of 850
building tree 145 of 850
building tree 146 of 850
building tree 147 of 850
building tree 148 of 850
building tree 149 of 850
building tree 150 of 850
building tree 151 of 850
building tree 152 of 850
building tree 153 of 850
building tree 154 of 850
building tree 155 of 850
building tree 156 of 850
building tree 157 of 850
building tree 158 of 850
building tree 159 of 850
building tree 160 of 850
building tree 161 of 850
building tree 162 of 850
building tree 163 of 850
building tree 164 of 850
building tree 165 of 850
building tree 166 of 850
building tree 167 of 850


...

building tree 512 of 850
building tree 513 of 850
building tree 514 of 850
building tree 515 of 850
building tree 516 of 850
building tree 517 of 850
building tree 518 of 850
building tree 519 of 850
building tree 520 of 850
building tree 521 of 850
building tree 522 of 850
building tree 523 of 850
building tree 524 of 850
building tree 525 of 850
building tree 526 of 850
building tree 527 of 850
building tree 528 of 850
building tree 529 of 850
building tree 530 of 850
building tree 531 of 850
building tree 532 of 850
building tree 533 of 850
building tree 534 of 850
building tree 535 of 850
building tree 536 of 850
building tree 537 of 850
building tree 538 of 850
building tree 539 of 850
building tree 540 of 850
building tree 541 of 850
building tree 542 of 850
building tree 543 of 850
building tree 544 of 850
building tree 545 of 850
building tree 546 of 850
building tree 547 of 850
building tree 548 of 850
building tree 549 of 850
building tree 550 of 850
building tree 551 of 850


[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed: 249.2min


building tree 800 of 850
building tree 801 of 850
building tree 802 of 850
building tree 803 of 850
building tree 804 of 850
building tree 805 of 850
building tree 806 of 850
building tree 807 of 850
building tree 808 of 850
building tree 809 of 850
building tree 810 of 850
building tree 811 of 850
building tree 812 of 850
building tree 813 of 850
building tree 814 of 850
building tree 815 of 850
building tree 816 of 850
building tree 817 of 850
building tree 818 of 850
building tree 819 of 850
building tree 820 of 850
building tree 821 of 850
building tree 822 of 850
building tree 823 of 850
building tree 824 of 850
building tree 825 of 850
building tree 826 of 850
building tree 827 of 850
building tree 828 of 850
building tree 829 of 850
building tree 830 of 850
building tree 831 of 850
building tree 832 of 850
building tree 833 of 850
building tree 834 of 850
building tree 835 of 850
building tree 836 of 850
building tree 837 of 850
building tree 838 of 850
building tree 839 of 850


[Parallel(n_jobs=1)]: Done  31 tasks      | elapsed:    1.3s
[Parallel(n_jobs=1)]: Done 127 tasks      | elapsed:    5.2s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:   11.1s
[Parallel(n_jobs=1)]: Done 511 tasks      | elapsed:   20.7s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:   30.8s


Mean Squared Error: 15.491566214254998
Root Mean Squared Error: 3.935932699406203
Euclidian distance Error: 4.478891762041696


In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt
import joblib  # Import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from joblib import dump

# Splitting the dataset into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Load the pre-trained RandomForest model
random_forest_model = joblib.load('/content/best_model.xgb')

'''# Set warm_start to True and increase the number of estimators
random_forest_model.set_params(warm_start=True, n_estimators=random_forest_model.n_estimators + 10)

# Fit the model again on the same data
random_forest_model.fit(X_train, Y_train)'''



df = pd.read_csv('/content/drive/MyDrive/evaluation.csv')



df = pre_process_removing_area_removing_outliers(df, ev=True)

    # predict the values using the df_clean
X = df.iloc[:, 1:]
y = df.iloc[:, :1]

    #use X to predict and print the prediction

y_pred = random_forest_model.predict(X)

    #convert y_pred to pandas dataframe

y_pred = pd.DataFrame(y_pred)
df_pred = pd.DataFrame()
df_pred['Id'] = y

df_pred['Predicted'] = y_pred.apply(lambda row: f"{row[0]} | {row[1]}", axis=1)


df_pred.to_csv('/content/drive/MyDrive/fullPipeline.csv', index=False)

# Predicting the targets
predictions = random_forest_model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print(f"Mean Squared Error: {mse}")
rmse = sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")
euclidian= euclidean_distance_error(Y_test, predictions)
print(f"Euclidian distance Error: {euclidian}")

In [ ]:
model = joblib.load('/content/drive/MyDrive/randmFrst4,7acc.joblib')

    #get evaluation csv and run preprocessing on it

df = pd.read_csv('/content/drive/MyDrive/evaluation.csv')



